In [454]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np


In [513]:
driver = webdriver.Chrome("/Applications/chromedriver")
driver.get(naturalia)

In [463]:
# get main shelter that sit on top of parent shelters => Main > Parent > Child

amazon_prime_now_stores = []

naturalia = "https://primenow.amazon.fr/storefront?merchantId=A14TQEU3FGQYTZ"
monoprix = "https://primenow.amazon.fr/storefront?merchantId=A39IAEDNN88TCS"
amazon_prime_now_stores.append(naturalia)
amazon_prime_now_stores.append(monoprix)


In [496]:
driver.get(naturalia)

In [478]:
# Naturalia
naturalia_urls = []

driver.get(naturalia)

try:
    page = driver.find_element_by_id('viewAllCitiesContent')
    onboarding_cities_links = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
    driver.get(onboarding_cities_links[1])
except:
    pass
    
top_categories = driver.find_elements_by_class_name('topCategory__container__2hxFv')[1:12]

for top_category in top_categories:
    top_cat_name = top_category.find_element_by_class_name('categoryTile__displayName__2WsVC').find_elements_by_tag_name('a')[0].find_element_by_css_selector("span").text
    sub_categories = top_category.find_elements_by_class_name('filter__subCategory__2HeRV')
    for sub_category in sub_categories:
        sub_cat_name = sub_category.find_elements_by_tag_name('a')[0].text
        link = sub_category.find_elements_by_tag_name('a')[0].get_attribute("href")
        naturalia_urls.append({"url":link, 
                             "shelve_parent": top_cat_name,
                             "shelve": sub_cat_name                                   
                            })
        
len(naturalia_urls)


55

In [482]:
# Monoprix
monoprix_urls = []

driver.get(monoprix)

    
top_categories = driver.find_elements_by_class_name('topCategory__container__2hxFv')[1:18]

for top_category in top_categories:
    top_cat_name = top_category.find_element_by_class_name('categoryTile__displayName__2WsVC').find_elements_by_tag_name('a')[0].find_element_by_css_selector("span").text
    sub_categories = top_category.find_elements_by_class_name('filter__subCategory__2HeRV')
    for sub_category in sub_categories:
        sub_cat_name = sub_category.find_elements_by_tag_name('a')[0].text
        link = sub_category.find_elements_by_tag_name('a')[0].get_attribute("href")
        monoprix_urls.append({"url":link, 
                             "shelve_parent": top_cat_name,
                             "shelve": sub_cat_name                                   
                            })
        
len(monoprix_urls)


106

In [519]:
amazon_urls = []
amazon_urls.append(naturalia_urls)
amazon_urls.append(monoprix_urls)
amazon_urls = [item for urls in amazon_urls for item in urls]

In [511]:
link = naturalia_urls[6]['url']
driver.get(link)
soup = BeautifulSoup(driver.page_source, 'lxml')
pagination = soup.select('.app__paginate-wrapper__2Ttq0 li')
next_pages = []
for element in pagination[2:-1]:
    try:
        next_pages.append('https://primenow.amazon.fr' + element.find("a")["href"])
    except:
        pass
next_pages



['https://primenow.amazon.fr/search?srs=17419833031&qid=1580471666&rh=p_95%3AU023&page=2&ref=sr_pg_2&ie=UTF8',
 'https://primenow.amazon.fr/search?srs=17419833031&qid=1580471666&rh=p_95%3AU023&page=3&ref=sr_pg_3&ie=UTF8']

In [520]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

product_pages = []

for data in amazon_urls:
    link = data['url']
    parent_shelve = data['shelve_parent']
    shelve = data['shelve']
 
    results = []
    brands = []

    driver.get(link)
  
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    results.append(soup.select('.product_grid__item__37iIU'))
    
    pagination = soup.select('.app__paginate-wrapper__2Ttq0 li')
    next_pages = []
    for element in pagination[2:-1]:
        try:
            next_pages.append('https://primenow.amazon.fr' + element.find("a")["href"])
        except:
            pass
    try:
        filters = driver.find_element_by_class_name('filterButton__filters-button-container__1d6s1').click() 
        brand_filter = driver.find_elements_by_class_name('index__filterSection__2nuyc li')
        brands = [elem.text for elem in brand_filter if len(elem.text) > 1]
    except:
        print(link)
        pass
    
    if len(next_pages) > 0:
        for page_link in next_pages:
            driver.get(page_link)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            results.append(soup.select('.product_grid__item__37iIU'))
            try:
                filters = driver.find_element_by_class_name('filterButton__filters-button-container__3N7lj').click()
                brand_filter = driver.find_elements_by_class_name('index__filterSection__2nuyc li')
                brands.append([elem.text for elem in brand_filter if len(elem.text) > 1])
            except:
                print(page_link)
                print(filters)
                pass
        
    flat_results = [item for subresults in results for item in subresults]


    
    product_pages.append({"url":link, 
                         "shelter_main": shelve,
                         "shelter_parent": parent_shelve,
                          "brands": brands,
                         "soup": flat_results
                        })

    



https://primenow.amazon.fr/browse?node=17419913031&merchantId=A14TQEU3FGQYTZ&ref_=pn_sf_ct_17419913031_1_A14TQEU3FGQYTZ&pf_rd_r=THBJQQG2AQAFH68ZXFQY&pf_rd_p=18351390-c777-4560-8e70-d2fc2f45a899&pf_rd_s=desktop-center-9&pf_rd_i=A14TQEU3FGQYTZ&pf_rd_t=101&pf_rd_m=AGMEOZFASZJSS
https://primenow.amazon.fr/browse?node=17489649031&merchantId=A14TQEU3FGQYTZ&ref_=pn_sf_ct_17489649031_2_A14TQEU3FGQYTZ&pf_rd_r=THBJQQG2AQAFH68ZXFQY&pf_rd_p=18351390-c777-4560-8e70-d2fc2f45a899&pf_rd_s=desktop-center-9&pf_rd_i=A14TQEU3FGQYTZ&pf_rd_t=101&pf_rd_m=AGMEOZFASZJSS
https://primenow.amazon.fr#
None
https://primenow.amazon.fr#
None
https://primenow.amazon.fr#
None
https://primenow.amazon.fr#
None
https://primenow.amazon.fr#
None
https://primenow.amazon.fr#
None


In [523]:
data = pd.DataFrame(product_pages)
data.head()

,url,shelter_main,shelter_parent,brands,soup
0,https://primenow.amazon.fr/browse?node=1741985...,Légumes frais,Fruits et légumes,"[Desconocido, Monoprix, NATURALIA, Naturalia, []]","[[[<a class=""asin_card__productLink__1-GHX"" da..."
1,https://primenow.amazon.fr/browse?node=1742008...,Fruits frais,Fruits et légumes,[[]],"[[[<a class=""asin_card__productLink__1-GHX"" da..."
2,https://primenow.amazon.fr/browse?node=1772741...,Panier de fruits et légumes,Fruits et légumes,[NATURALIA],"[[[<a class=""asin_card__productLink__1-GHX"" da..."
3,https://primenow.amazon.fr/browse?node=2048060...,Vrac,Fruits et légumes,"[ACCENT BIO SARL, DAMIANO, LOU PRUNEL SARL, MO...","[[[<a class=""asin_card__productLink__1-GHX"" da..."
4,https://primenow.amazon.fr/browse?node=1741985...,Fruits secs,Fruits et légumes,"[COULEUR QUEBEC, DAMIANO, Ethnoscience, LES AC...","[[[<a class=""asin_card__productLink__1-GHX"" da..."


In [526]:
import re

products = []
bad_products = []

for page in product_pages:
    shelter_main = page['shelter_main']
    shelter_parent = page['shelter_parent']
    brands = [brand for brand in page['brands'] if not '[]']
    
    for content in page['soup']:
        for product in content:
            product_name = product.find("div", class_="asin_card__title__TAkef").text
            try:
                product_img = product.find("img")['src']
            except:
                bad_products.append(product)
                break
            product_link = 'https://primenow.amazon.fr' + product.find("a", class_="asin_card__productLink__1-GHX")["href"]
            product_asin = re.search(r"(?<=/dp/)[^,]+", product_link).group()[:10]
            store_name = product.find("div", class_="asin_offer_price__root__nXLh8").find("img")["alt"]
            price = product.find("div", class_="asin_price__priceFull__27h7K").text
            try:
                price_per_unit = product.find("div", class_="asin_price__pricePerUnit__3zFwH").text
            except:
                pass
            try:
                promo_price = product.find("div", class_="asin_price__priceDiscount__Ug2dH").text
            except:
                pass
            if len(brands) > 0:
                brand = ''.join([brand for brand in brands if brand in product_name])
            else:
                brand = ""  
            clean_description = product_name.lower().strip().replace(brand.lower().strip(), '').strip()

            products.append({"shelter_main": shelter_main,
                             "shelter_parent": shelter_parent,
                             "brand": brand,  
                             "description1": clean_description,                             
                             "description": product_name,
                             "price": price,
                             "price_per_unit":price_per_unit,
                             "promo_price": promo_price,
                             "store": store_name,
                             "image_url": product_img,                    
                             "url": product_link,
                             "ean": product_asin
                            })
print(len(products))
print(len(bad_products))


3530
223


In [527]:
product_data = pd.DataFrame(products)
product_data = product_data.drop_duplicates()

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_amazonprimenow.csv', index = None, header=True)


In [528]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = product_data[product_data.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [shelter_main, shelter_parent, brand, description1, description, price, price_per_unit, promo_price, store, image_url, url, ean]
Index: []
